In [45]:
import torch 
import torchvision 
import numpy as np

import os
import pandas as pd
import albumentations as A
import albumentations.pytorch
import cv2

from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
# import torch.nn.functional as F

import sys
sys.path.append('../')
from dataset import val_transforms, CDataset

In [46]:
import easydict 
args = easydict.EasyDict({ "batch_size": 64, 
                          "epochs": 60, 
                          "data": 0, 
                          'lr':0.02,
                         'momentum':0.9,
                         'weight_decay':1e-4,
                         'start_epoch':0,
                         'gpu':1,
                          'workers':16,
                         'print_freq':2000,
                         'saved_dir':'../trained_models/checkpoint.pt'})

In [47]:
# make saved dir
from pathlib import Path
path = Path(args.saved_dir.split('checkpoint')[0])
path.mkdir(parents=True, exist_ok=True)

In [48]:
ngpus_per_node = torch.cuda.device_count()
print(ngpus_per_node)
GPU_NUM = args.gpu # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)
print(device)

3
cuda:1


In [49]:
from augment import *

In [50]:
IMAGE_SIZE = 256
train_transforms = A.Compose([
    A.Resize(IMAGE_SIZE, IMAGE_SIZE, p=1),
    A.OneOf([
        A.HorizontalFlip(p=.8),
        A.VerticalFlip(p=.8),
        A.RandomRotate90(p=.8)]
    ),
    A.OneOf([
        HEColorAugment(sigma1=.2, sigma2=2., mat=None, p=0.9),
        HEColorAugment(sigma1=.4, sigma2=5., mat=None, p=0.9),
        HEColorAugment(sigma1=.6, sigma2=5., mat=None, p=0.9)
    ]),
], p=1.0) 

In [51]:
import albumentations as A
import albumentations.pytorch
import cv2

class CDataset(Dataset):
    def __init__(self, df, defaultpath='/home/beomgon/Dataset/new_patches/', transform=None):
        self.df = df
        self.transform = transform
        self.dir = defaultpath

    def __len__(self):
        return len(self.df)   

    def __getitem__(self, idx):
        path = self.df.iloc[idx, 4]
#         print(pid)

        image = cv2.imread(self.dir + path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#         image = (image.astype(np.float32)-128.)/128.
        
#         if image is uint8, normalization by 255 is done automatically by albumebtation(ToTensor method)
        if self.transform:
            timage = self.transform(image=image)
            image = timage['image']
        
        image =  torch.tensor(image, dtype=torch.float32)/255.
        #image = (torch.tensor(image, dtype=torch.float32)-128)/128
        image = image.permute(2,0,1)
            
        label = self.df.iloc[idx, 5]
        return image, label, path

In [52]:
train_df = pd.read_csv('../dataframe/train_New_Stomach_df.csv')
train_dataset = CDataset(train_df, defaultpath='/home/beomgon/Dataset/seegene/new_patches/', 
                         transform=train_transforms)

val_df = pd.read_csv('../dataframe/val_New_Stomach_df.csv')
val_dataset = CDataset(val_df, defaultpath='/home/beomgon/Dataset/seegene/new_patches/',
                       transform=val_transforms)  

test_df = pd.read_csv('../dataframe/test_New_Stomach_df.csv')
test_dataset = CDataset(test_df, defaultpath='/home/beomgon/Dataset/seegene/new_patches/',
                        transform=val_transforms)  

In [53]:
image, label, path = next(iter(train_dataset))
# image

In [54]:
train_loader = DataLoader(train_dataset, batch_size=args.batch_size,
                          shuffle=True, num_workers=args.workers)

val_loader = DataLoader(val_dataset, batch_size=args.batch_size,
                          shuffle=True, num_workers=args.workers)

test_loader = DataLoader(test_dataset, batch_size=args.batch_size,
                          shuffle=True, num_workers=args.workers)


In [55]:
# images, labels, paths = next(iter(train_loader))
# images.shape

In [56]:
# from resnet import resnet18
# model = resnet18(pretrained=False)
# checkpoint = torch.load('../trained_models/checkpoint.pth')
# # model.load_state_dict(checkpoint['state_dict'], strict=False)

In [57]:
# import re
# new_checkpoint = {}
# for i in checkpoint['state_dict'].keys() :
#     n = re.sub('module.','',i)
#     new_checkpoint[n] = checkpoint['state_dict'][i]

# model.load_state_dict(new_checkpoint, strict=False)

In [58]:
# 

In [59]:
# import torchvision.models as models
from resnet import resnet152, resnet18
model = resnet18(pretrained=False)
# model.load_state_dict(new_checkpoint, strict=False)
model.fc = nn.Linear(32, 3)
torch.nn.init.normal_(model.fc.weight, std=0.01)
torch.nn.init.constant_(model.fc.bias, 0.)
model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
# optimizer = torch.optim.Adam(params, lr=0.01, weight_decay=args.weight_decay)
optimizer = torch.optim.SGD(
       params, lr=args.lr, momentum=args.momentum, weight_decay=args.weight_decay)
# optimizer = torch.optim.SGD(params

























































































































6
, lr=args.lr)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10, 20, 30, 40, 50], 
                                                    gamma=0.2)
# criterion = nn.CrossEntropyLoss(label_smoothing=0.0)
criterion = nn.CrossEntropyLoss().to(device)

In [60]:
from main import *

In [61]:
ratio = [.8, .8, .8]
acc_list = []
test_acc = []

for num in range(len(ratio)) :
    args.saved_dir = '../trained_models/resnet8/cus_stomach4_' + str(num) + '/checkpoint.pt'
    path = Path(args.saved_dir.split('checkpoint')[0])
    path.mkdir(parents=True, exist_ok=True)  
    
    model = resnet18(pretrained=False)
    # model.load_state_dict(new_checkpoint, strict=False)
    # model.fc.out_features = 3
    model.fc = nn.Linear(32, 3)
    torch.nn.init.normal_(model.fc.weight, std=0.01)
    torch.nn.init.constant_(model.fc.bias, 0.)    
    # model.load_state_dict(default_state_dict)
    model.to(device)

    params = [p for p in model.parameters() if p.requires_grad]
#     optimizer = torch.optim.Adam(params, lr=0.01, weight_decay=args.weight_decay)
    # optimizer = torch.optim.SGD(
    #        params, lr=args.lr, momentum=args.momentum, weight_decay=args.weight_decay)
    optimizer = torch.optim.SGD([
                    {'params': model.conv1.parameters(), 'lr': args.lr},
                    {'params': model.bn1.parameters(), 'lr': args.lr},
                    {'params': model.layer1.parameters(), 'lr': args.lr*2},
                    {'params': model.layer2.parameters(), 'lr': args.lr*2},
                    {'params': model.layer3.parameters(), 'lr': args.lr*3},
                    {'params': model.layer4.parameters(), 'lr': args.lr*3},
                    {'params': model.layer5.parameters(), 'lr': args.lr*4},
                    {'params': model.fc.parameters(), 'lr': args.lr*4},
                ], lr=args.lr, momentum=args.momentum, weight_decay=args.weight_decay)    
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10, 15, 20, 25], 
                                                        gamma=0.2)
    # criterion = nn.CrossEntropyLoss(label_smoothing=0.0)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_acc1 = 0
    acc1 = 0
    train_loss = []
    val_acc = []

    for epoch in range(args.epochs):
        print('lr', optimizer.param_groups[0]['lr'], optimizer.param_groups[2]['lr'])
        losses = train(train_loader, model, criterion, optimizer, epoch, args)
        scheduler.step()

        # evaluate after every epoch
        acc1 = validate(val_loader, model, criterion, args)   
        val_acc.append(acc1.item())

        # remember best acc@1 and save checkpoint
        is_best = acc1 > best_acc1
        best_acc1 = max(acc1, best_acc1) 

        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'best_acc1': best_acc1,
            'optimizer' : optimizer.state_dict(),
            'scheduler' : scheduler.state_dict(),
        }, is_best, filename=args.saved_dir)   
        
    checkpoint = torch.load(args.saved_dir)
    model.load_state_dict(checkpoint['state_dict'])
    acc1 = validate(test_loader, model, criterion, args)    
    print('************* test number {} ratio {} acc{} *************'.format(num, ratio[num], acc1))    
    acc_list.append(val_acc)    
    test_acc.append(acc1)
    

lr 0.02 0.04
Epoch: [0][   0/1185]	Time  2.708 ( 2.708)	Data  2.618 ( 2.618)	Loss 1.0983e+00 (1.0983e+00)	Acc@1  35.94 ( 35.94)
Test: [  0/406]	Time  1.215 ( 1.215)	Loss 6.1871e-01 (6.1871e-01)	Acc@1  71.88 ( 71.88)
 * Acc@1 71.758
lr 0.02 0.04
Epoch: [1][   0/1185]	Time  3.650 ( 3.650)	Data  3.600 ( 3.600)	Loss 6.3301e-01 (6.3301e-01)	Acc@1  78.12 ( 78.12)
Test: [  0/406]	Time  1.051 ( 1.051)	Loss 4.8643e-01 (4.8643e-01)	Acc@1  79.69 ( 79.69)
 * Acc@1 82.836
lr 0.02 0.04
Epoch: [2][   0/1185]	Time  2.378 ( 2.378)	Data  2.299 ( 2.299)	Loss 3.8005e-01 (3.8005e-01)	Acc@1  85.94 ( 85.94)
Test: [  0/406]	Time  1.123 ( 1.123)	Loss 8.2151e-01 (8.2151e-01)	Acc@1  78.12 ( 78.12)
 * Acc@1 71.719
lr 0.02 0.04
Epoch: [3][   0/1185]	Time  3.530 ( 3.530)	Data  3.468 ( 3.468)	Loss 2.8821e-01 (2.8821e-01)	Acc@1  85.94 ( 85.94)
Test: [  0/406]	Time  1.154 ( 1.154)	Loss 2.6239e-01 (2.6239e-01)	Acc@1  92.19 ( 92.19)
 * Acc@1 84.985
lr 0.02 0.04
Epoch: [4][   0/1185]	Time  2.572 ( 2.572)	Data  2.521 ( 2.

KeyboardInterrupt: 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

epochs = range(args.epochs)
plt.rcParams["figure.figsize"] = (8,6)
plt.rcParams['lines.linewidth'] = 1
plt.plot(epochs, acc_list[0], linestyle='--')
plt.plot(epochs, acc_list[1])
# plt.plot(epochs, acc_list[2])
# plt.plot(epochs, acc_list[3])
plt.legend(['.4', '.4', '.4' ])
plt.xlabel('epochs')
plt.ylabel('val acc')
plt.title('val acc with stain augment using resnet18 for stomach')
# plt.axis([10, 30, 88, 94])
plt.show()
print(test_acc)

In [ ]:
args.saved_dir = '../trained_models/resnet8/cus_stomach4_' + str(1) + '/checkpoint.pt'
checkpoint = torch.load(args.saved_dir)
model.load_state_dict(checkpoint['state_dict'])
acc1 = validate(test_loader, model, criterion, args)    

In [ ]:
# optimizer = torch.optim.SGD([
#                 {'params': model.conv1.parameters(), 'lr': args.lr*4},
#                 {'params': model.bn1.parameters(), 'lr': args.lr*4},
#                 {'params': model.layer1.parameters(), 'lr': args.lr*2},
#                 {'params': model.layer2.parameters(), 'lr': args.lr*2},
#                 {'params': model.layer3.parameters(), 'lr': args.lr*1},
#                 {'params': model.layer4.parameters(), 'lr': args.lr*1},
#                 {'params': model.fc.parameters(), 'lr': args.lr*1},
#             ], lr=args.lr, momentum=args.momentum, weight_decay=args.weight_decay)   

In [ ]:
# optimizer.param_groups[6]['lr']
model.fc.weight.requires_grad = True
model.fc.bias.requires_grad = True

In [ ]:
from main import *

model = resnet18(pretrained=False, linear_evaluation=True)
model.fc = nn.Linear(16, 3)
args.saved_dir = '../trained_models/resnet8/cus_stomach4_' + str(1) + '/checkpoint.pt'
checkpoint = torch.load(args.saved_dir)
model.load_state_dict(checkpoint['state_dict'])
model.train()
for p in model.parameters() :
    p.requires_grad = False
model.fc.weight.requires_grad = True
model.fc.bias.requires_grad = True
model.to(device)

best_acc1 = 0
acc1 = 0

optimizer = torch.optim.Adam(params, lr=0.0001, weight_decay=args.weight_decay)

acc1 = validate(val_loader, model, criterion, args)   
print('acc1', acc1)

for i in range(10) :
    for i, (images, targets, _) in enumerate(train_loader):

        if args.gpu is not None:
            images = images.cuda(args.gpu, non_blocking=True)
        if torch.cuda.is_available():
            targets = targets.cuda(args.gpu, non_blocking=True)

        # compute output
        output = model(images)
        loss = criterion(output, targets)    
        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()    
        
    # evaluate after every epoch
    acc1 = validate(val_loader, model, criterion, args)   

    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1) 

    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
        'scheduler' : scheduler.state_dict(),
    }, is_best, filename=args.saved_dir)           